## Bronze to Silver Transformation

In [0]:
from pyspark.sql.functions import from_utc_timestamp,date_format
from pyspark.sql.types import TimestampType

table_name = []
for i in dbutils.fs.ls('/mnt/bronze/SalesLT/'):
    table_name.append(i.name.split('/')[0])

for i in table_name:
    path = '/mnt/bronze/SalesLT/'+i+'/'+i+'.parquet'
    df = spark.read.format('parquet').load(path)
    column = df.columns
    for col in column:
        if 'date' in col or 'Date' in col :
            df = df.withColumn(col,date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"),"yy-MM-dd"))
    output_path = '/mnt/silver/'+i+'/'
    df.write.format('delta').mode("overwrite").save(output_path)